In [12]:
from transformers import pipeline
from datasets import load_dataset

# Dataset

In [4]:
dataset_checkpoint = "kde4"
dataset_commit_id = "12cd06d961fae220f6ef1ab533321b8e9ddc3533"

In [5]:
raw_datasets = load_dataset(dataset_checkpoint, lang1="en", lang2="fr", revision=dataset_commit_id)

In [6]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 210173
    })
})

## Split dataset

In [9]:
split_datasets = raw_datasets["train"].train_test_split(train_size=0.9, seed=20)
split_datasets["validation"] = split_datasets.pop("test")

In [10]:
split_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 189155
    })
    validation: Dataset({
        features: ['id', 'translation'],
        num_rows: 21018
    })
})

# Model

In [13]:
model_checkpoint = "Helsinki-NLP/opus-mt-en-fr"
model_commit_id = "0b11e64c9efac19014daf61fb333354e09000f00"

In [14]:
translator = pipeline("translation", model=model_checkpoint, revision=model_commit_id)

## Translation differences
The pre-trained model tends to use less formal translations (e.g., keeps certain english terms in the translation)

In [11]:
split_datasets["train"][1]["translation"]

{'en': 'Default to expanded threads',
 'fr': 'Par défaut, développer les fils de discussion'}

In [16]:
translator(split_datasets["train"][1]["translation"]["en"])

[{'translation_text': 'Par défaut pour les threads élargis'}]

In [18]:
split_datasets["train"][172]["translation"]

{'en': 'Unable to import %1 using the OFX importer plugin. This file is not the correct format.',
 'fr': "Impossible d'importer %1 en utilisant le module d'extension d'importation OFX. Ce fichier n'a pas un format correct."}

In [19]:
translator(split_datasets["train"][172]["translation"]["en"])

[{'translation_text': "Impossible d'importer %1 en utilisant le plugin d'importateur OFX. Ce fichier n'est pas le bon format."}]